# Single Table Classifier

This first tutorial trains a classifier on a single table dataset.

In [52]:
import pandas as pd
from khiops.sklearn import KhiopsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score

## Training a Classifier

We'll train a classifier for the `Iris` dataset. This is a classical dataset containing data of different plants belonging to the genus _Iris_. It contains 150 records, 50 for each of the three _Iris_'s variants: _Setosa_, _Virginica_ and _Versicolor_. Each record contains the length and the width of both the petal and the sepal of the plant. The standard task, when using this dataset, is to construct a classifier for the type of the _Iris_, based on the petal and sepal characteristics.

To train a classifier with Khiops, we only need a dataframe that we are going to load from a file. 

In [5]:
# Load data directly from our GitHub
url = "https://raw.githubusercontent.com/KhiopsML/khiops-samples/main/Iris/Iris.txt"

# Read the data into a Pandas DataFrame
iris_df = pd.read_csv(url, delimiter='\t')

# Display the first 10 records from the dataset
iris_df.head(10)

,SepalLength,SepalWidth,PetalLength,PetalWidth,Class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
5,5.4,3.9,1.7,0.4,Iris-setosa
6,4.6,3.4,1.4,0.3,Iris-setosa
7,5.0,3.4,1.5,0.2,Iris-setosa
8,4.4,2.9,1.4,0.2,Iris-setosa
9,4.9,3.1,1.5,0.1,Iris-setosa


Before training the classifier, we split the data into the feature matrix (sepal length, width, etc) and the target vector containing the labels (the `Class` column).

In [39]:
# Drop the "class" column to create the feature set (X).
X_iris = iris_df.drop("Class", axis=1)
# Extract the "class" column to create the target labels (y).
y_iris = iris_df["Class"]

Then we can construct our final train / test dataset

In [40]:
# Build our train and test dataset
X_iris_train, X_iris_test, y_iris_train, y_iris_test = train_test_split(X_iris, y_iris)

Let's check the contents of the feature matrix and the target vector:

In [41]:
# Features of the Iris dataset
X_iris_train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth
23,5.1,3.3,1.7,0.5
9,4.9,3.1,1.5,0.1
130,7.4,2.8,6.1,1.9
101,5.8,2.7,5.1,1.9
86,6.7,3.1,4.7,1.5


In [42]:
#Labels of the Iris datase
y_iris_train.unique()

array(['Iris-setosa', 'Iris-virginica', 'Iris-versicolor'], dtype=object)

Let's now train the classifier with the pyKhiops function `KhiopsClassifier`. This method returns a model ready to classify new Iris plants.

In [46]:
pkc_iris = KhiopsClassifier()
pkc_iris.fit(X_iris_train, y_iris_train)

KhiopsClassifier()

## Accessing the Classifier' Basic Train Evaluation Metrics

Khiops calculates evaluation metrics for the training dataset. We access them via the model's attribute `model_report` which is an instance of the `AnalysisResults` class. Let's check this out:

In [47]:
iris_train_performance = pkc_iris.model_report_.train_evaluation_report.get_snb_performance()

This object `iris_train_performance` is of class `PredictorPerformance` and has `accuracy` and `auc` attributes:

In [49]:
print(f"Iris train accuracy: {iris_train_performance.accuracy}")
print(f"Iris train AUC     : {iris_train_performance.auc}")

Iris train accuracy: 0.973214
Iris train AUC     : 0.998921


The `PredictorPerformance` object has also a confusion matrix attribute:

In [50]:
iris_classes = iris_train_performance.confusion_matrix.values
iris_confusion_matrix = pd.DataFrame(
    iris_train_performance.confusion_matrix.matrix,
    columns=iris_classes,
    index=iris_classes,
)
print("Iris train confusion matrix:")
iris_confusion_matrix

Iris train confusion matrix:


,Iris-setosa,Iris-versicolor,Iris-virginica
Iris-setosa,37,0,0
Iris-versicolor,0,36,0
Iris-virginica,0,3,36


## Deploying a Classifier
We are now going to deploy the `Iris` classifier `pkc_iris`, that we have just trained.

The learned classifier can be deployed in two different ways:

- to predict a class that can be obtained using the `predict` method of the model.
- to predict class probabilities that can be obtained using the `predict_proba` method of the model.

Let's first predict the `Iris` labels:

In [51]:
iris_predictions = pkc_iris.predict(X_iris_test)
print("Iris model predictions (first 10 values):")
iris_predictions[:10]

Iris model predictions (first 10 values):


array(['Iris-virginica', 'Iris-setosa', 'Iris-virginica',
       'Iris-versicolor', 'Iris-virginica', 'Iris-versicolor',
       'Iris-versicolor', 'Iris-versicolor', 'Iris-setosa', 'Iris-setosa'],
      dtype='<U15')

From these predictions we can compute the accuracy score using `sklearn.metrics`

In [55]:
# from sklearn.metrics
accuracy_score(y_iris_test, iris_predictions)

0.8947368421052632

Let's now predict the probabilities for each `Iris` type.
Note that the column order of this matrix is given by the estimator attribute `pkc.classes_`:

In [45]:
iris_probas = pkc_iris.predict_proba(X_iris_test)
print(f"Iris classes {pkc_iris.classes_}")
print("Iris model probabilities for each class (first 10 rows):")
iris_probas[:10]

Iris classes ['Iris-setosa' 'Iris-versicolor' 'Iris-virginica']
Iris model probabilities for each class (first 10 rows):


array([[0.00213819, 0.07063872, 0.92722309],
       [0.99571261, 0.00214709, 0.0021403 ],
       [0.00213819, 0.07063872, 0.92722309],
       [0.00213366, 0.98828409, 0.00958225],
       [0.00213819, 0.07063872, 0.92722309],
       [0.00433708, 0.89626777, 0.09939515],
       [0.00213366, 0.98828409, 0.00958225],
       [0.00213366, 0.98828409, 0.00958225],
       [0.99571261, 0.00214709, 0.0021403 ],
       [0.99571261, 0.00214709, 0.0021403 ]])

Then, we can compute a ROC_AUC score using `sklearn.metrics` setting the multi_class parameter

In [58]:
# from sklearn.metrics
# Calculate the ROC-AUC score using the One-vs-Rest approach
roc_auc_score(y_iris_test, iris_probas, multi_class='ovr')

0.9820276174442841